In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

transactions_products = transactions.merge(products)
merged_data = transactions_products.merge(customers)

if 'Price_x' in merged_data.columns and 'Price_y' in merged_data.columns:
    merged_data = merged_data.drop(columns=['Price_y'])
    merged_data.rename(columns={'Price_x': 'Price'})

merged_data.columns = merged_data.columns.str.strip()

customer_agg = merged_data.groupby("CustomerID").agg({
    "Price": "mean",
    "ProductID": "count",
    "Region": "first"
}).reset_index()
customer_agg.rename(columns={"Price": "AvgSpending", "ProductID": "TransactionCount"}, inplace=True)

customer_agg = pd.get_dummies(customer_agg, columns=["Region"], drop_first=True)

scaler = StandardScaler()
customer_agg[["AvgSpending", "TransactionCount"]] = scaler.fit_transform(customer_agg[["AvgSpending", "TransactionCount"]])

similarity_matrix = cosine_similarity(customer_agg.drop(columns=["CustomerID"]))
similarity_df = pd.DataFrame(similarity_matrix, index=customer_agg["CustomerID"], columns=customer_agg["CustomerID"])

def get_similar_customers(target_customer_id, top_n=3):
    if target_customer_id not in similarity_df.index:
        print(f"CustomerID {target_customer_id} not found.")
        return
    similar_customers = similarity_df[target_customer_id].sort_values(ascending=False)[1:top_n+1]
    result = customer_agg[customer_agg["CustomerID"].isin(similar_customers.index)]
    result["SimilarityScore"] = similar_customers.values
    return result
def main():
    target_customer_id = input("enter CustomerID to find similar users: ").capitalize()
    similar_customers = get_similar_customers(target_customer_id)
    if similar_customers is not None:
        print("\nTop similar customers:")
        print(similar_customers[["CustomerID", "SimilarityScore"]])

if __name__ == "__main__":
    main()
